# molecular descriptors for ALDH1 inhibitors:

Different descriptors can be determined on the given molecules to find a (causal) relation between the molecule and the ability to inhibit ALDH1. 

Molecular descriptors are for instance: 
-   molecular mass
-   nr carbon atoms
-   nr hydrogen atoms  
-   nr of bonds
-   nr of branches
-   nr double bindings
-   nr triple bindings
-   cyclic structures
-   Aromaticity (indicated by lower letters)
    -   aromatic nitrogen
-   (tetra hedral) chirality
- nr of rings (e.g. cubane)

### rdkit has automatic implemented descriptors and Fingerprints:

This is used now for the generation of descriptors. also a couple of fingerprint variables can be included. 


In [165]:
import numpy as np
import pandas as pd
from tkinter import filedialog as fd
from rdkit import Chem, DataStructs
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw, Descriptors, AllChem
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
IPythonConsole.ipython_useSVG=True



In [166]:
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, cohen_kappa_score, matthews_corrcoef
import joblib

In [167]:
filename = fd.askopenfilename()
AHDL1Inhibitors = pd.read_csv(filename ,header = None)

In [168]:
allTestedMolecules = AHDL1Inhibitors[0] # firts 3 for testing, needs to change for all molecules (remove[0:4])
MolList = allTestedMolecules.values.tolist()
with open('AllTestedMols.txt', 'w') as fp:
    for mol in MolList:
        # write each item on a new line
        fp.write("%s\n" % mol)
    print('Done')

Done


### Generate Fingerprints

In [169]:
#x = all data from Principal components 1:61
x = AHDL1Inhibitors.iloc[:,2:-2]
x.shape


(1001, 60)

In [170]:

#y is inhibition score 
# check imbalance dataset:
y = AHDL1Inhibitors.iloc[:,-1]
#sum(y)/len(y)
print(y)
# mild imbalance

0       ALDH1_inhibition
1                      1
2                      1
3                      1
4                      1
              ...       
996                    0
997                    0
998                    0
999                    0
1000                   0
Name: 63, Length: 1001, dtype: object


Use data for Machine Learning: 
- Random forest
- SVM

In [171]:
# split data in train and test sets. Set the testset size to 20%
seed = 13
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.20, random_state=seed)
# create folds for cross- validation
cv = StratifiedKFold(n_splits=5, shuffle=True,random_state=seed)

In [172]:
# print out ids of folds
for i, (train_index, test_index) in enumerate(cv.split(xTrain, yTrain)):
    print("\nFold_" + str(i+1))
    print("TRAIN:", train_index)
    print("TEST:", test_index)


Fold_1
TRAIN: [  0   1   2   3   4   5   7   8   9  11  12  13  14  15  16  17  19  20
  21  22  23  25  26  28  29  30  31  32  33  34  35  37  38  39  40  41
  42  43  44  45  47  49  50  51  52  53  54  55  57  58  59  60  61  62
  63  64  65  67  69  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  93  94  95  96  98 100 101 102 103 104 105
 106 107 108 109 110 112 114 115 117 118 119 121 122 123 124 125 128 129
 133 134 136 137 138 139 140 141 142 143 144 145 146 147 148 149 152 153
 154 155 156 157 158 159 160 161 162 164 165 166 167 168 170 171 172 173
 175 176 177 178 180 181 182 183 184 185 186 188 189 190 191 192 193 195
 196 198 199 200 201 202 205 206 207 208 209 210 211 212 214 215 216 217
 218 219 221 223 224 225 227 228 229 230 231 234 235 237 238 240 241 242
 243 244 245 246 250 251 252 253 254 255 257 258 259 260 262 263 264 268
 270 271 272 273 277 278 280 281 282 284 286 287 288 289 290 293 295 297
 298 299 301 303 304 305 306 307 308

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [173]:
# Scale inputs
scale = StandardScaler().fit(xTrain)
xTrain = scale.transform(xTrain)


In [174]:
# save data for future use
#joblib.dump(scale, "Fingerprints.pkl", compress=3)

In [175]:
# create grid search dictionary
param_grid = {"max_features": [xTrain.shape[1] // 10, xTrain.shape[1] // 7, 
                               xTrain.shape[1] // 5, xTrain.shape[1] // 3], 
              "n_estimators": [100, 250, 500]}

In [176]:
# setup model building
m = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=2, cv=cv, verbose=1)

In [177]:
# run model building
m.fit(xTrain, yTrain)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=13, shuffle=True),
             estimator=RandomForestClassifier(), n_jobs=2,
             param_grid={'max_features': [6, 8, 12, 20],
                         'n_estimators': [100, 250, 500]},
             verbose=1)

In [178]:
m.best_params_

{'max_features': 6, 'n_estimators': 250}

In [179]:
m.best_score_

0.70125

In [180]:
m.cv_results_


{'mean_fit_time': array([0.31276617, 0.73453741, 1.47186823, 0.34679322, 0.86981702,
        1.75028458, 0.47187119, 1.18227906, 2.34577532, 0.69628825,
        1.73974137, 3.43618832]),
 'std_fit_time': array([0.01415045, 0.00023926, 0.00618651, 0.00624976, 0.00553957,
        0.01004707, 0.01152084, 0.03734588, 0.03784192, 0.00708817,
        0.0191408 , 0.04535987]),
 'mean_score_time': array([0.01249547, 0.03124781, 0.0499886 , 0.01249704, 0.03125691,
        0.0531137 , 0.00937066, 0.02320061, 0.05268636, 0.00799923,
        0.02993846, 0.04998546]),
 'std_score_time': array([6.24773561e-03, 1.51911368e-05, 6.24938025e-03, 6.24852276e-03,
        1.55144376e-05, 7.64860217e-03, 7.65111257e-03, 6.41179107e-03,
        5.94629755e-03, 3.18946676e-06, 2.99930678e-03, 6.24904637e-03]),
 'param_max_features': masked_array(data=[6, 6, 6, 8, 8, 8, 12, 12, 12, 20, 20, 20],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, 

In [181]:
joblib.dump(m, "RFmodelMorganFingerprint.pkl", compress=3)

['RFmodelMorganFingerprint.pkl']

Load model (also to check if it works)

In [182]:
scale = joblib.load("Fingerprints.pkl")
# scale descriptors of the test set compounds
xTest = scale.transform(xTest)
# predict logBB class
predRF = m.predict(xTest)
predRF

array(['0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0',
       '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0',
       '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0

In [183]:
accuracy_score(yTest, predRF)

0.7213930348258707

In [184]:
# if the model includes several ones like RF models or consensus models (or for probabilistic models)
# we can calculate consistency of predictions amongs those models and use it for estimation of applicability domain
pred_prob = m.predict_proba(xTest)
pred_prob

array([[0.64 , 0.36 , 0.   ],
       [0.644, 0.356, 0.   ],
       [0.732, 0.268, 0.   ],
       [0.752, 0.248, 0.   ],
       [0.652, 0.348, 0.   ],
       [0.488, 0.512, 0.   ],
       [0.66 , 0.34 , 0.   ],
       [0.624, 0.376, 0.   ],
       [0.54 , 0.46 , 0.   ],
       [0.704, 0.296, 0.   ],
       [0.476, 0.524, 0.   ],
       [0.904, 0.096, 0.   ],
       [0.744, 0.256, 0.   ],
       [0.724, 0.276, 0.   ],
       [0.828, 0.172, 0.   ],
       [0.712, 0.288, 0.   ],
       [0.676, 0.324, 0.   ],
       [0.612, 0.388, 0.   ],
       [0.872, 0.128, 0.   ],
       [0.644, 0.356, 0.   ],
       [0.664, 0.336, 0.   ],
       [0.636, 0.364, 0.   ],
       [0.82 , 0.18 , 0.   ],
       [0.696, 0.304, 0.   ],
       [0.348, 0.652, 0.   ],
       [0.796, 0.204, 0.   ],
       [0.72 , 0.28 , 0.   ],
       [0.648, 0.352, 0.   ],
       [0.668, 0.332, 0.   ],
       [0.792, 0.208, 0.   ],
       [0.796, 0.204, 0.   ],
       [0.672, 0.328, 0.   ],
       [0.696, 0.304, 0.   ],
       [0.

In [185]:
# setup threshold
threshold = 0.8
# calc maximum predicted probability for each row (compound) and compare to the threshold
da = np.amax(pred_prob, axis=1) > threshold
da

array([False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False,  True, False, False, False,
        True, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False,  True, False,
       False,  True,  True,  True, False,  True, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False,  True, False, False,  True, False, False, False,
       False,  True, False, False, False, False,  True, False, False,
       False,  True, False, False, False, False, False,  True, False,
       False, False,

In [186]:
# calc statistics
accuracy_score(np.asarray(yTest)[da], predRF[da])

0.9333333333333333

In [187]:
# calc coverage
sum(da) / len(da)

0.14925373134328357

SVM approach

In [188]:
# create grid search dictionary
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}
# setup model building
svm = GridSearchCV(SVC(kernel='rbf', probability=True), param_grid, n_jobs=2, cv=cv, verbose=1)

In [189]:
# run model building
svm.fit(xTrain, yTrain)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=13, shuffle=True),
             estimator=SVC(probability=True), n_jobs=2,
             param_grid={'C': [1, 10, 100, 1000, 10000],
                         'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]},
             verbose=1)

In [190]:
svm.best_params_

{'C': 1, 'gamma': 1e-06}

In [191]:
svm.best_score_

0.69375

In [192]:
# save model
joblib.dump(svm, "SVMmorganFingerprint.pkl", compress=3)

['SVMmorganFingerprint.pkl']

In [193]:
# predict logBB for the test set compounds
pred_svm = svm.predict(xTest)
pred_svm


array(['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0

In [194]:
# calc statistics
print("Accuracy = ", accuracy_score(yTest, pred_svm))
print("MCC = ", matthews_corrcoef(yTest, pred_svm))
print("Kappa = ", cohen_kappa_score(yTest, pred_svm))

Accuracy =  0.7213930348258707
MCC =  0.0
Kappa =  0.0


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:870: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [214]:
# estimate applicability domain and calc stat
pred_prob = svm.predict_proba(xTest[:,:60])
pred_prob

array([[0.59116231, 0.40626088, 0.00257681],
       [0.61521108, 0.38254407, 0.00224485],
       [0.66225049, 0.33606647, 0.00168304],
       [0.60819847, 0.38942382, 0.00237771],
       [0.59913075, 0.39682796, 0.00404129],
       [0.59065881, 0.34836716, 0.06097403],
       [0.63617895, 0.36127642, 0.00254462],
       [0.53566665, 0.46202641, 0.00230694],
       [0.41767707, 0.57835545, 0.00396748],
       [0.65973902, 0.33830859, 0.00195239],
       [0.54805609, 0.44991924, 0.00202467],
       [0.6549944 , 0.34341722, 0.00158839],
       [0.66408057, 0.3314328 , 0.00448663],
       [0.62457978, 0.37373847, 0.00168175],
       [0.66056148, 0.33786404, 0.00157448],
       [0.66635166, 0.33183445, 0.00181389],
       [0.6109874 , 0.38744055, 0.00157205],
       [0.65167364, 0.3464404 , 0.00188596],
       [0.68346825, 0.31519498, 0.00133677],
       [0.65251384, 0.34510069, 0.00238547],
       [0.57235596, 0.42551925, 0.00212478],
       [0.64613756, 0.3517715 , 0.00209094],
       [0.

In [196]:
da = np.amax(pred_prob, axis=1) > threshold

In [197]:
print("Accuracy = ", accuracy_score(np.asarray(yTest)[da], pred_svm[da]))
print("MCC = ", matthews_corrcoef(np.asarray(yTest)[da], pred_svm[da]))
print("Kappa = ", cohen_kappa_score(np.asarray(yTest)[da], pred_svm[da]))
print("Coverage = ", sum(da) / len(da))

Accuracy =  nan
MCC =  0.0
Kappa =  nan
Coverage =  0.0


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:870: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:638: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)


Add the descriptors to the model

In [198]:
#xNew = np.concatenate((x,allDescrsDf), axis=1)

xNew = x

In [199]:
xNtr, xNts, yNtr, yNts = train_test_split(xNew, y, test_size=0.20, random_state=seed)
scale = StandardScaler().fit(xNtr)
xNtr = scale.transform(xNtr)

In [200]:
# create grid search dictionary
param_grid = {"max_features": [xNtr.shape[1] // 10, xNtr.shape[1] // 7, xNtr.shape[1] // 5, xNtr.shape[1] // 3], 
              "n_estimators": [100, 250, 500]}

In [201]:
# setup model building
m = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=2, cv=cv, verbose=1)

In [202]:
# run model building
m.fit(xNtr, yNtr)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=13, shuffle=True),
             estimator=RandomForestClassifier(), n_jobs=2,
             param_grid={'max_features': [6, 8, 12, 20],
                         'n_estimators': [100, 250, 500]},
             verbose=1)

In [203]:
m.best_score_

0.70125

In [204]:
# scale descriptors of the test set compounds
xNts = scale.transform(xNts)
# predict
pred = m.predict(xNts)
pred

array(['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0',
       '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0',
       '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0

In [205]:
# calc statistics
print("Accuracy = ", accuracy_score(yNts, pred))
print("MCC = ", matthews_corrcoef(yNts, pred))
print("Kappa = ", cohen_kappa_score(yNts, pred))

Accuracy =  0.7164179104477612
MCC =  0.09442748954272834
Kappa =  0.06358806702084197


In [206]:
# estimate applicability domain and calc stat
pred_prob = m.predict_proba(xNts)
da = np.amax(pred_prob, axis=1) > threshold

print("Accuracy = ", accuracy_score(np.asarray(yNts)[da], pred[da]))
print("MCC = ", matthews_corrcoef(np.asarray(yNts)[da], pred[da]))
print("Kappa = ", cohen_kappa_score(np.asarray(yNts)[da], pred[da]))
print("Coverage = ", sum(da) / len(da))

Accuracy =  0.9393939393939394
MCC =  0.0
Kappa =  0.0
Coverage =  0.16417910447761194


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:870: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [207]:
# rebuild RF model manually using best parameters to be able to extract additional information from the model
rf = RandomForestClassifier(n_estimators=m.best_params_["n_estimators"], 
                           max_features=m.best_params_["max_features"],
                           random_state=seed)
rf.fit(xNtr, yNtr)

RandomForestClassifier(max_features=8, n_estimators=250, random_state=13)

In [208]:
imp = rf.feature_importances_
imp

array([0.0283915 , 0.01506679, 0.01666252, 0.01455994, 0.02256103,
       0.0167347 , 0.01321682, 0.01767279, 0.0196165 , 0.01629196,
       0.01605649, 0.02187571, 0.02164369, 0.0135019 , 0.01550466,
       0.01774237, 0.01446973, 0.01847863, 0.01700047, 0.01514201,
       0.0157182 , 0.01884253, 0.01677849, 0.0156523 , 0.01294984,
       0.0144665 , 0.02108312, 0.01529573, 0.01664576, 0.03053783,
       0.01506413, 0.01430257, 0.01484031, 0.02310789, 0.01550658,
       0.01473334, 0.01569203, 0.014062  , 0.02050756, 0.01806977,
       0.01391965, 0.01577274, 0.01179744, 0.01485772, 0.01455075,
       0.01937181, 0.01476011, 0.01224036, 0.01469677, 0.01480179,
       0.01393039, 0.01627655, 0.01833641, 0.0142722 , 0.01540484,
       0.01385946, 0.01951104, 0.01614332, 0.01627379, 0.01317615])

In [209]:
indices = np.argsort(imp)[::-1]

print("Feature ranking:")

# print top 10 features
for i in range(10):
    print("%d. feature %d (%f)" % (i + 1, indices[i], imp[indices[i]]))

Feature ranking:
1. feature 29 (0.030538)
2. feature 0 (0.028391)
3. feature 33 (0.023108)
4. feature 4 (0.022561)
5. feature 11 (0.021876)
6. feature 12 (0.021644)
7. feature 26 (0.021083)
8. feature 38 (0.020508)
9. feature 8 (0.019616)
10. feature 56 (0.019511)


features 1-2028 are different Morgan fingerprints